Simple Moving Average, with different windows shows the effects of smoothing

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ticker_symbol = "AMZN"
start = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
today = datetime.now().strftime("%Y-%m-%d")

stock_data = yf.download(ticker_symbol, start=start, end=today)

# Print the fetched data
print(stock_data.head())


# plot
plt.figure(figsize=(15, 7))

plt.xlabel("Time")
plt.xlabel("Price")

stock_data["Adj Close"].plot(linewidth=2.0)

stock_data["SMA-3"] = stock_data["Adj Close"].rolling(3).mean()
stock_data["SMA-3"].plot(linewidth=2.0)

stock_data["SMA-10"] = stock_data["Adj Close"].rolling(10).mean()
stock_data["SMA-10"].plot(linewidth=2.0)

stock_data["SMA-25"] = stock_data["Adj Close"].rolling(30).mean()
# stock_data["SMA-25"].plot(linewidth=2.0)
columns = ["Close","SMA-3", "SMA-10", "SMA-25"]
plt.legend(columns, prop={'size': 15})
plt.show()